In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
df_visits = pd.read_excel('BI_Analyst_Assessment_Data_2023.xlsx',sheet_name='Visit Tbl')
df_invoice = pd.read_excel('BI_Analyst_Assessment_Data_2023.xlsx',sheet_name='Invoice Tbl')
df_diag = pd.read_excel('BI_Analyst_Assessment_Data_2023.xlsx',sheet_name='Diagnosis Tbl')

In [2]:
df_visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45226 entries, 0 to 45225
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   VisitCode      45226 non-null  object        
 1   PatientCode    45226 non-null  object        
 2   VisitDateTime  45226 non-null  datetime64[ns]
 3   MedicalCenter  45226 non-null  object        
 4   VisitCategory  45226 non-null  object        
 5   Payor          45226 non-null  object        
 6   NPS Score      1928 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 2.4+ MB


In [3]:
df_invoice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46005 entries, 0 to 46004
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   VisitCode  46005 non-null  object
 1   Amount     46005 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 719.0+ KB


In [4]:
df_diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21860 entries, 0 to 21859
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   VisitCode  21860 non-null  object
 1   Diagnosis  21860 non-null  object
dtypes: object(2)
memory usage: 341.7+ KB


In [5]:
visits_invoice_merge.merge(df_diag,on='VisitCode',how='left').info()

NameError: name 'visits_invoice_merge' is not defined

1. How many visits did Kimathi Street and Pipeline medical centers' have from May 2022 and September 2022?

In [6]:
df_visits['MedicalCenter'].value_counts()

Tassia            11177
Pipeline           9519
Call Centre        5236
Embakasi           5168
Githurai 45        4166
Lucky Summer       3387
Kimathi Street     3349
Mathare North      3224
Name: MedicalCenter, dtype: int64

In [7]:
kimath_pipe_visits = df_visits[(df_visits['MedicalCenter']=='Kimathi Street')| (df_visits['MedicalCenter']=='Pipeline')]
kimathi_pipe_march_sep_visits = kimath_pipe_visits[(kimath_pipe_visits['VisitDateTime']>='2022-05-01') & (kimath_pipe_visits['VisitDateTime']<'2022-09-1')]
kimathi_pipe_march_sep_visits.shape

(6730, 7)

In [8]:
kimath_pipe_visits.VisitDateTime.dt.month_name().value_counts()

June         1996
May          1723
October      1687
March        1625
July         1575
September    1474
August       1436
April        1352
Name: VisitDateTime, dtype: int64

In [9]:
kimath_pipe_visits = df_visits[(df_visits['MedicalCenter']=='Kimathi Street')| (df_visits['MedicalCenter']=='Pipeline')]
kimath_pipe_visits_march = kimath_pipe_visits[(kimath_pipe_visits['VisitDateTime']>'2022-05-01') & (kimath_pipe_visits['VisitDateTime']<'2022-05-31')]
kimath_pipe_visits_sept = kimath_pipe_visits[(kimath_pipe_visits['VisitDateTime']>'2022-09-01') & (kimath_pipe_visits['VisitDateTime']<'2022-09-30')]
kimathi_pipe_march_sep_visits = pd.concat([kimath_pipe_visits_sept,kimath_pipe_visits_march])
kimathi_pipe_march_sep_visits.shape

(3086, 7)

In [10]:
kimath_pipe_visits_sept.shape + kimath_pipe_visits_march.shape

(1420, 7, 1666, 7)

2. What was the most common diagnosis in 2022 for Tassia and Embakasi branches combined?

In [11]:
visit_diag_merge = pd.merge(df_visits,df_diag,on='VisitCode',how='left')
tass_emb_selec = visit_diag_merge[(visit_diag_merge['MedicalCenter']=='Tassia') | (visit_diag_merge['MedicalCenter']=='Embakasi')]
tass_emb_selec['Diagnosis'].value_counts()

acute nasopharyngitis                  1512
tonsillitis, acute bacterial           1317
review                                  814
acute gastritis                         620
acute rhinitis                          618
acute bronchitis                        504
family planning                         474
normal pregnancy                        422
pneumonia                               418
gastroenteritis with no dehydration     362
myalgia                                 267
soft tissue injury                      236
food poisoning                          229
dermatitis                              164
migraine                                127
common cold                             111
acute pharyngitis, viral                 86
amoebiasis                               86
allergic rhinitis                        86
cystitis                                 78
acute otitis media                       77
eczema                                   66
bacterial conjunctivitis        

In [12]:
visit_diag_merge.groupby(['MedicalCenter','Diagnosis']).agg({'Diagnosis':'count'})

Diagnosis
MedicalCenter Diagnosis                              
Call Centre   acute bronchitis                     11
              acute gastritis                      85
              acute nasopharyngitis                94
              acute otitis media                    5
              acute pharyngitis, viral             10
...                                               ...
Tassia        septic wound                         23
              severe malaria                       22
              severe pneumonia                     24
              soft tissue injury                  149
              tonsillitis, acute bacterial        936

[242 rows x 1 columns]

3. Which payor was the most profitable (in absolute numbers) for Penda Health in 2022? {Assume a gross average margin of 30% per visit}.

In [13]:
insurance = df_visits[df_visits['Payor']!='Cash']
insurace_visits = list(insurance['VisitCode'].unique())
df_invoice_clean = df_invoice.copy()
mask = (df_invoice_clean['VisitCode'].isin(insurace_visits)) & (df_invoice_clean['Amount']<100)
df_invoice_clean.loc[mask,'Amount'] = 100
df_invoice_clean = df_invoice_clean.groupby('VisitCode').agg({'Amount':'sum'}).reset_index()

In [14]:
insur_inv_merge = df_visits.merge(df_invoice_clean,how='left',on='VisitCode')
insur_inv_merge['Profit'] = insur_inv_merge['Amount'] * 0.3
insur_inv_merge.groupby('Payor').agg({'Profit':'sum'}).sort_values(by='Profit',ascending=False)

,Profit
Payor,
Cash,11269575.0
Insurance Company B,10051335.9
Insurance Company A,1669599.6


4. Which medical center was the least profitable  (in absolute numbers)  in 2022?  {Assume a gross average margin of 30% per visit}.

In [15]:
# inv_visit_merge = df_visits.merge(df_invoice,how='left',on='VisitCode')

# # the least possible invoice amount for any insurance visit should be 100
# inv_visit_merge['Amount'].iloc[(inv_visit_merge['Amount']==0) & (inv_visit_merge['Payor']!='Cash')] = 100
# inv_visit_merge['Profit'] = inv_visit_merge['Amount'] * 0.3
insur_inv_merge.groupby('Payor').agg({'Profit':'sum'}).sort_values(by='Profit',ascending=False)
insur_inv_merge.groupby('MedicalCenter').agg({'Profit':'sum'}).sort_values(by='Profit')

,Profit
MedicalCenter,
Call Centre,617146.8
Mathare North,1487595.3
Githurai 45,1976979.9
Lucky Summer,2002607.1
Kimathi Street,2571794.4
Embakasi,2753515.2
Pipeline,5716678.5
Tassia,5864193.3


5. What was the average spend per visit for visits that had a diagnosis of acute gastritis?

In [16]:
df_diag_clean = df_diag.copy()
df_diag_clean = df_diag_clean.groupby('VisitCode')['Diagnosis'].apply(lambda x: ','.join(x)).reset_index()
df_diag_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19736 entries, 0 to 19735
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   VisitCode  19736 non-null  object
 1   Diagnosis  19736 non-null  object
dtypes: object(2)
memory usage: 308.5+ KB


In [17]:
visit_inv_merge = df_visits.merge(df_invoice_clean,how='left',on='VisitCode')
diag_inv_merge = visit_inv_merge.merge(df_diag_clean,on='VisitCode',how='inner')
diag_inv_merge[diag_inv_merge['Diagnosis'].str.contains('acute gastritis')]['Amount'].mean()
# diag_inv_merge[diag_inv_merge['Diagnosis']=='acute gastritis']['Amount'].mean()

2820.2234375

6. At Penda Health, we have a blended healthcare model where patients can get treatment physically (by visiting a medical centers) or virtually (through our call center). In 2022, how many unique patients experienced a blended healthcare approach in their healthcare journey?

In [18]:
telemed_patients = df_visits[df_visits['VisitCategory']=='Telemedicine Visit']['PatientCode'].unique()
len(df_visits[(df_visits['VisitCategory']=='In-person Visit')&(df_visits['PatientCode']).isin(telemed_patients)]['PatientCode'].unique())

284

7. Calculate the Net Promoter Score (NPS) in Q3 2022. {Please note that valid NPS scores range from 0 to 10}

In [19]:
df_visits[df_visits['VisitDateTime'].dt.quarter == 3]['NPS Score'].mean()

8.932203389830509

8. In 2022 what proportion of visits in Penda Health were second visits?{A first visit is when a patient visits Penda Health for the very first time, a second visit is when the same patient visits again i.e. a second time and so on}

In [20]:
(df_visits[(df_visits.groupby('PatientCode').cumcount() == 1)].shape[0] / df_visits.shape[0])*100

18.40313094237828

9. What percent of visits in April 2022 happened within 30 days of the preceding visit by the same patient?

In [21]:
mar_apr_visits = df_visits[(df_visits['VisitDateTime']>'2022-03-01')&(df_visits['VisitDateTime']<'2022-04-30')]
mar_apr_visits = mar_apr_visits.groupby('PatientCode')['VisitDateTime'].apply(lambda x: x.sort_values()).reset_index()

In [22]:
mar_apr_visits = df_visits[(df_visits['VisitDateTime']>'2022-03-01')&(df_visits['VisitDateTime']<'2022-04-30')]
mar_apr_visits = mar_apr_visits.sort_values(by='VisitDateTime').copy()
mar_apr_visits['diff']= mar_apr_visits.groupby('PatientCode')['VisitDateTime'].diff() / np.timedelta64(1, 'D')
mar_apr_visits['diff'] = mar_apr_visits['diff'].fillna(0)

In [23]:
(mar_apr_visits[(mar_apr_visits['diff']>0)&(mar_apr_visits['diff']<=30)].shape[0] / df_visits.shape[0])*100

3.9999115553000486

In [24]:
mar_apr_visits['diff'] = mar_apr_visits.drop(columns='level_1',axis=1)['VisitDateTime'].diff() / np.timedelta64(1, 'D')
mar_apr_visits['diff'] = mar_apr_visits['diff'].round()
mar_apr_visits[mar_apr_visits['diff']>1]

KeyError: "['level_1'] not found in axis"

In [ ]:
mar_apr_visits

,PatientCode,level_1,VisitDateTime
0,0006f3bf-6789,44377,2022-03-03 19:39:17
1,00073e0c-8a3a,18702,2022-04-14 13:13:59
2,000def8c-b89d,26600,2022-03-07 15:33:34
3,000def8c-b89d,26322,2022-03-08 09:51:04
4,00118793-a612,44563,2022-03-26 21:25:08
...,...,...,...
9384,fff3ab14-0644,32364,2022-04-27 17:54:13
9385,fff3afc9-fc31,24088,2022-03-03 18:03:48
9386,fff3afc9-fc31,23739,2022-03-08 14:42:54
9387,fff3afc9-fc31,25624,2022-03-09 17:51:44


In [28]:
df_visits[df_visits['NPS Score'].notna()].describe()

,NPS Score
count,1928.000000
mean,8.872407
std,2.468548
min,0.000000
25%,9.000000
50%,10.000000
75%,10.000000
max,11.000000
